### **Phase 1: Player Detection and Initial Analysis** 🏃‍♂️⚽️

1. **YOLOv11 Fine-Tuning for Player Detection** 🎯
   - Adapt YOLOv11 for detecting football players on the field.
   - **Objective**: Ensure robust detection of players, even under challenging scenarios like crowded scenes or varying lighting conditions.

In [ ]:
import torch
print(torch.backends.mps.is_available())
print(torch.cuda.is_available())

False
True


In [10]:
!yolo task=detect mode=train model="/content/drive/MyDrive/FootCVision/utils/yolo11s.pt" data="/content/drive/MyDrive/dataset/data.yaml" epochs=10 imgsz=640 device=0

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/FootCVision/utils/yolo11s.pt, data=/content/drive/MyDrive/dataset/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cro

1.2 **Inference on Youtube Video** !

In [2]:
!yolo task=detect mode=predict model="/Users/alyazouzou/Desktop/CV_Football/FootCVision2/phase1/runs/detect/train/weights/best.pt" source="/Users/alyazouzou/Desktop/CV_Football/vids/demo.mov" device=mps project="/Users/alyazouzou/Desktop/CV_Football/vids" name="output"

Ultralytics 8.3.57 🚀 Python-3.9.21 torch-2.5.1 MPS (Apple M3)
YOLO11s summary (fused): 238 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs

video 1/1 (frame 1/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo.mov: 384x640 1 goalkeeper, 17 players, 1 referee, 188.4ms
video 1/1 (frame 2/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo.mov: 384x640 1 goalkeeper, 17 players, 1 referee, 26.0ms
video 1/1 (frame 3/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo.mov: 384x640 1 goalkeeper, 18 players, 1 referee, 14.1ms
video 1/1 (frame 4/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo.mov: 384x640 1 goalkeeper, 18 players, 1 referee, 15.7ms
video 1/1 (frame 5/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo.mov: 384x640 1 goalkeeper, 18 players, 1 referee, 14.6ms
video 1/1 (frame 6/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo.mov: 384x640 1 goalkeeper, 18 players, 1 referee, 19.0ms
video 1/1 (frame 7/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo

2. **Conformal Object Detection with puncc library** 📏

In [7]:
import os
import glob
import numpy as np
from PIL import Image
from inference import PlayerInference
from odwrapper import YOLOAPIWrapper

from deel.puncc.object_detection import SplitBoxWise
from deel.puncc.metrics import object_detection_mean_coverage, object_detection_mean_area
from ultralytics import YOLO
from sklearn.model_selection import train_test_split

In [8]:
dataset_path = "/Users/alyazouzou/Desktop/CV_Football/dataset"
train_images_path = os.path.join(dataset_path, "train/images")
train_labels_path = os.path.join(dataset_path, "train/labels")

def parse_yolo_dataset(images_path, labels_path, percentage=0.01, random_seed=42):
    """
    Parse a subset of the YOLO dataset (e.g., 1%) and extract image paths, ground truth boxes, and labels.
    Args:
        images_path (str): Path to the images folder.
        labels_path (str): Path to the labels folder.
        percentage (float): Percentage of data to parse (e.g., 0.01 for 1%).
        random_seed (int): Random seed for reproducibility.
    Returns:
        Tuple[List[str], List[np.ndarray], List[List[int]]]: Subsampled image paths, ground truth boxes, and labels.
    """
    # Use glob to get all .txt files
    label_files = glob.glob(os.path.join(labels_path, "*.txt"))

    # Set random seed for reproducibility
    np.random.seed(random_seed)

    # Determine the number of files to sample
    total_files = len(label_files)
    sample_size = int(percentage * total_files)

    # Randomly select a subset of files
    sampled_files = np.random.choice(label_files, size=sample_size, replace=False)

    image_paths = []
    ground_truth_boxes = []
    all_labels = []

    for label_file in sampled_files:
        with open(label_file, "r") as f:
            labels = f.readlines()

        # Extract bounding boxes and associated class IDs
        boxes = []
        labels_per_image = []
        for label in labels:
            parts = list(map(float, label.strip().split()))
            class_id, x_center, y_center, width, height = parts
            image_file = os.path.join(images_path, os.path.basename(label_file).replace(".txt", ".jpg"))
            image = Image.open(image_file)
            image_width, image_height = image.size
            x_center *= image_width
            y_center *= image_height
            width *= image_width
            height *= image_height
            # Convert YOLO format to bounding box (x1, y1, x2, y2)
            x1 = x_center - (width / 2)
            y1 = y_center - (height / 2) #ici c'est pas + ? 
            x2 = x_center + (width / 2)
            y2 = y_center + (height / 2) #et ici - ? 
            boxes.append([x1, y1, x2, y2])
            labels_per_image.append(int(class_id))

        # Add the image path, boxes, and labels
        image_file = os.path.join(images_path, os.path.basename(label_file).replace(".txt", ".jpg"))
        image_paths.append(image_file)
        ground_truth_boxes.append(boxes)
        all_labels.append(labels_per_image)

    return image_paths, ground_truth_boxes, all_labels


# Parse the YOLO dataset
image_paths, ground_truth_boxes, all_labels = parse_yolo_dataset(train_images_path, train_labels_path)

# Convert to numpy arrays for compatibility
X = np.array(image_paths)
y = np.array(ground_truth_boxes, dtype=object)  # Use dtype=object for variable-length arrays
labels = np.array(all_labels, dtype=object)  # Use dtype=object for variable-length arrays

In [9]:
yolo_apii = YOLOAPIWrapper(model_path="/Users/alyazouzou/Desktop/CV_Football/FootCVision/phase1/runs/detect/train/weights/best.pt")

2 0.096875 0.44296875 0.0203125 0.1328125
2 0.196875 0.37890625 0.021875 0.115625
2 0.1734375 0.62265625 0.0421875 0.140625
2 0.2265625 0.31953125 0.0234375 0.09375
2 0.29921875 0.365625 0.028125 0.1015625
2 0.39140625 0.29375 0.0203125 0.096875
3 0.496875 0.48671875 0.025 0.1234375
2 0.43828125 0.5515625 0.028125 0.128125
2 0.57890625 0.621875 0.028125 0.1296875
2 0.64140625 0.36875 0.0265625 0.0859375
2 0.8359375 0.28984375 0.0234375 0.0953125
2 0.446875 0.2390625 0.0203125 0.0921875
3 0.36484375 0.20859375 0.025 0.0890625
0 0.59296875 0.40234375 0.0125 0.0234375
[
x
1
,
y
1
,
x
2
,
y
2
]
, where 
(
x
1
,
y
1
)
 represents the top-left corner and 
(
x
2
,
y
2
)
 represents the bottom-right corner of the predicted bounding box.

In [ ]:
print(yolo_apii.predict_and_match(X[0], y[0]))

In [13]:
from deel.puncc.api.prediction import IdPredictor
from deel.puncc.object_detection import SplitBoxWise

# Create the proxy for the object detection model
api_model = IdPredictor()

In [11]:
# Appel à la méthode query
y_preds, y_trues_matched, images, classes = yolo_apii.query(X, y, labels)

print("Predictions:", y_preds)
print("Matched Ground Truths:", y_trues_matched)
print("Images:", images)
print("Classes:", classes)

Predictions: [[     54.713      242.29       69.63      322.99]
 [     119.49      207.96      131.28      279.33]
 [     97.053       355.1      124.25      443.97]
 ...
 [        184      379.78      192.71      417.41]
 [     211.67      375.17      222.93      408.38]
 [     49.366      214.69      56.337      239.69]]
Matched Ground Truths: [[       55.5         241        68.5         326]
 [        119       205.5         133       279.5]
 [       97.5       353.5       124.5       443.5]
 ...
 [      183.5         379       193.5         417]
 [        213         376         223         409]
 [         50         215          57         241]]
Images: ['/Users/alyazouzou/Desktop/CV_Football/dataset/train/images/324_jpg.rf.b1c72e3f0c6559a99e19500d21bc8f72.jpg', '/Users/alyazouzou/Desktop/CV_Football/dataset/train/images/Image251_png_jpg.rf.d0f588eaa5097f4e5be8767cc27a5ad3.jpg', '/Users/alyazouzou/Desktop/CV_Football/dataset/train/images/Image154_png_jpg.rf.59bcd343205743d34c0cd1

In [14]:
# Instantiate the conformal predictor
conformal_predictor = SplitBoxWise(api_model, method="multiplicative", train=False)

# Fit the conformal predictor, directly provide the predicted bboxes rather than image inputs
conformal_predictor.fit(X_calib=y_preds, y_calib=y_trues_matched)

### Inference on one Test Image

In [ ]:
# Select an image from the test dataset
image, bboxes, classes = test_dataset[1]

# Predict on the image
y_new_api = object_detection_api.predict_from_image(image)

# Choose the coverage target 1-alpha
alpha = 0.3

# Inference + UQ
y_pred_new, box_inner, box_outer = api_cp.predict(y_new_api, alpha=alpha)

In [ ]:
from deel.puncc.plotting import draw_bounding_box

image_with_bbox = image.copy()

for i in range(len(y_pred_new)):
    image_with_bbox = draw_bounding_box(
        image=image_with_bbox,
        box=bboxes[i],
        label=classes[i],
        legend="Truth",
        color="red",
    )
    image_with_bbox = draw_bounding_box(
        image=image_with_bbox,
        box=y_pred_new[i],
        legend="Predictions",
        color="blue",
    )
    image_with_bbox = draw_bounding_box(
        image=image_with_bbox,
        box=box_outer[i],
        legend="Conformalized Outer Box",
        color="orange",
    )
    
    # image_with_bbox = draw_bounding_box(
    #     image=image_with_bbox,
    #     box=box_inner[i],
    #     legend="Conformalized Inner Box",
    #     color="brown",
    # )

_ = draw_bounding_box(image=image_with_bbox, show=True)

### Inference on All Test Image

In [ ]:
# Evaluate the results
coverage = object_detection_mean_coverage(box_outer, y_test)
average_area = object_detection_mean_area(box_outer)

print(f"Marginal coverage: {np.round(coverage, 2)}")
print(f"Average area of prediction intervals: {np.round(average_area, 2)}")